<div>
  <div>
    Emme Notebook and Scripting Course, August 2019 <br>
  </div>
  <div>
    <img style="align: left; margin: 15px 15px 15px 0px;" src="./INRO Logo.png" width="120" />
  </div>
  <div>
    Â© Copyright 2019 INRO
  </div>
</div>

# 3 Matrix API - Read, Write and Process Matrix Data
This class provides examples of Matrix API use to access, manipulate, import and export matrix data. 

__Suggested Duration__: 2 hours

## 3.1 Contents 

<a href="#3.1-Contents">3.1 Contents</a>

<a href="#3.2-Accessing-the-Matrix-API">3.2 Accessing the Matrix API</a>

<a href="#3.3-Working-with-MatrixData-objects">3.3 Working with MatrixData objects</a>
- <a href="#3.3.1-Matrix-calculations">3.3.1 Matrix calculations</a>
- <a href="#3.3.2-PRACTICE:-Matrix-API">3.3.2 PRACTICE: Matrix API</a>
- <a href="#3.3.3-Matrix-manipulations-(transpose,-submatrix,-expansion)">3.3.3 Matrix manipulations  (transpose, submatrix, expansion)</a>

<a href="#3.4-Working-with-NumPy-arrays">3.4 Working with NumPy arrays</a>

<a href="#3.5-Importing-and-exporting-to-CSV-format">3.5 Importing and exporting to CSV format</a>

<a href="#3.6-Additional-Example-part-1:-Modify-values-of-a-matrix-by-district">3.6 Additional Example part 1: Modify values of a matrix by district</a>


<a href="#3.7-Additional-Example-part-2:-Compute-weighted-avg.-travel-time-by-OD-pair">3.7 Additional Example part 2: Compute weighted avg. travel time by OD pair</a>


## 3.2 Accessing the Matrix API
The entry point for the Matrix API is through an Emme <code><b>Matrix</b></code> object, accessed by the <code><b>inro.emme.database.emmebank.Emmebank</b></code> class. Emme has four matrix types: scalar, origin, destination and full matrices. Scalar matrices are single values, origin and destination matrices are one-dimensional vectors containing origin or destination indices, and full matrices are two-dimensional matrices with both origin and destination indices. The following code will return the IDs of all full matrices currently defined in the Emme database:

In [ ]:
import inro.modeller as _m
modeller = _m.Modeller()
emmebank = modeller.emmebank
scenario = modeller.scenario

for matrix in emmebank.matrices():
    if matrix.type == "FULL":
        print matrix.id

To return an instance of a specific matrix, we can either reference it by its ID or its name. If using matrix names, these must be unique, or the <code>emmebank<b>.matrix()</b></code> method will return `None`. 

In [ ]:
matrix_mf4 = emmebank.matrix("mf4")
matrix_mfautsb = emmebank.matrix("mfautsb")

matrix_mf4 == matrix_mfautsb

Note that in the case that a matrix does not exist you may see an error such as <code>AttributeError: 'NoneType' object has no attribute 'name'</code>. You can check if the matrix does not exist by checking if the matrix lookup was <code>None</code>.

From the Database API we can also create new matrices, delete matrices, and lookup an available matrix ID. Below are two common workflows for managing matrix creation when the matrix may already exist. The first workflow consists in looking up a matrix by its ID and create it if it does not exists in the Emme bank. The second workflow consists in getting the next available matrix identifier and create a new matrix.

In [ ]:
# Lookup a matrix by ID, and if it does not exist create it
matrix_mf13 = emmebank.matrix("mf13")
if matrix_mf13 is None:
    matrix_mf13 = emmebank.create_matrix("mf13")
matrix_mf13.name = "Custom"

# Lookup a matrix by name, and if it does exist delete it.
matrix_custom = emmebank.matrix("mfCustom")
if matrix_custom:
    emmebank.delete_matrix(matrix_mf13.id)
    
# Create a new matrix using the next available identifier
new_matrix_id = emmebank.available_matrix_identifier("FULL")
matrix_custom = emmebank.create_matrix(new_matrix_id)
matrix_custom.name = "Custom2"

To access and manipulate the matrix data, the <code>matrix<b>.get_data()</b></code> method is used to return a <code><b>MatrixData</b></code> object, which is an in-memory copy of the matrix data:

In [ ]:
auto_times_data = matrix_mfautsb.get_data()

<b>Tip</b>: <code><b>MatrixData</b></code> references the data values by zone ID, which are determined by the zone system definition in the Emme database. If all scenarios share the same zone system (have identical centroid IDs), then it is not necessary to identify a particular scenario. However, if different scenarios have different zone systems, then a scenario ID must be provided to <code>matrix<b>.get_data(</b>scenario_id<b>)</b></code>, or you will see the following error <code>Error: Not all scenarios share the same zone system</code>.

In [ ]:
auto_times_data = matrix_mfautsb.get_data(scenario_id=3001)

To query the matrix data, use the <code><b>get()</b></code> method. The matrix values are referenced by the zone IDs, which need not be continuous. For example, to get the value at origin <code>p=5</code> and destination <code>q=7</code> for matrix <code>mf4</code>, use the following code:

In [ ]:
value = auto_times_data.get(5, 7)
print "The auto times (mf4) matrix value at O-D=(5,7) is", value

It is also possible to modify the data through assignment using the <code><b>set()</b></code> method. To change the value at origin <code>p=5</code> and destination <code>q=7</code> to 55.4, use the following code:

In [ ]:
auto_times_data.set(5, 7, 55.4)
new_value = auto_times_data.get(5, 7)

print "The new auto times (mf4) matrix value at O-D=(5,7) is", new_value

Once all operations are completed on the matrix data, we can write this data to the Emme <code><b>Matrix</b></code> object. We do so by using the <code><b>set_data()</b></code> method on the Emme matrix we want to write to:

In [ ]:
matrix_mfautsb.set_data(auto_times_data)

The previous operations are also available on scalar, origin and destination matrices. For origin and destination matrices, a single value is specified for the __`get()`__ and __`set()`__ methods, as shown in the following example:

In [ ]:
matrix_mo3 = emmebank.matrix("mo3")
mo3_data = matrix_mo3.get_data()
mo3_val_10  = mo3_data.get(10)
print "The mo3 matrix value at origin=10 is", mo3_val_10

To access the single value contained in a scalar matrix, use the __`data`__ attribute:

In [ ]:
matrix_ms2 = emmebank.matrix("ms2")
ms2_val = matrix_ms2.data
print "The value of scalar matrix ms2 is", ms2_val

## 3.3 Working with MatrixData objects

### 3.3.1 Matrix calculations
Calculations can be performed by iterating (<code>for</code> loop) through the zone IDs with the <code><b>get()</b></code> method to lookup values and the <code><b>set()</b></code> method to change values. The following example doubles the value of the auto demand if the current value is between 5 and 20. Any series of Python expressions can be used to calculate the new value, and multiple matrices can be calculated at the same time without saving intermediate values.

In [ ]:
auto_demand_matrix = emmebank.matrix("mfauds")
auto_demand = auto_demand_matrix.get_data()
zones = scenario.zone_numbers

for p in zones:
    for q in zones:
        value = auto_demand.get(p, q)
        if 5 <= value <= 20:
            auto_demand.set(p, q, value * 2)

The matrix data can be very efficient when changing values for a subset of zones as only the zones of interest need to be evaluated. A set of `selected_zones` can be identified from a <b><code>Partition</code></b> as shown in the example below.

Partition data is accessed through the same interface __`get_data()`__ as matrix data and __`get()`__, __`set()`__ functions to access the values.

In [ ]:
auto_demand_matrix = emmebank.matrix("mfauds")
auto_demand = auto_demand_matrix.get_data()

# The selected zones which are in the partition group 1, 2, or 3 (ga1, ga2, or ga3)
partition_data = emmebank.partition('ga').get_data()
selected_zones = []
for z in zones:
    if partition_data.get(z) in (1, 2, 3):
        selected_zones.append(z)

for p in selected_zones:
    for q in selected_zones:
        value = auto_demand.get(p, q)
        auto_demand.set(p, q, value * 2)

<p>
Note that in the above example the list of `selected_zones` was built in a separate `for` loop than that used for the <code><b>MatrixData</b></code> calculation. It is important to minimize the complexity of the "inner loop" expressions as this is run `zones**2` times.
</p>
<p>
General calculations with the Matrix Calculator tool or NumPy are normally faster than <code><b>MatrixData</b></code> `for` loops, however the later can be a useful option when using small fixed submatrices (`selected_zones`), calculations with intermediate values which are discarded, complex logical expressions which are easier to express in Python, or combined with the <code><b>transposed()</b></code>, <code><b>submatrix()</b></code> or <code><b>expand()</b></code> methods discussed under <a href="#Matrix-manipulations">Matrix manipulations</a> below.
</p>

<p>
    As one more example of useful application of the <code><b>MatrixData</b></code> `for` loop, below is a calculation which sets the intra-zonal values of a matrix. Note that there is only one `for` loop needed to accomplish this.
</p>

In [ ]:
# Set the intra-zonal demand to 0
for z in zones:
    auto_demand.set(z, z, 0)

Note that as long as we did not call the `matrix.set_data()` function, none of these modifications is published to the Emmebank.

### 3.3.2 <span style="color:red">PRACTICE: Matrix API</span>

Please refer to the _Emme Notebook and Scripting - Practices_ Notebook to complete this exercise. Note that the solutions to practices are found in the _Emme Notebook and Scripting - Solutions_ Notebook.

### 3.3.3 Matrix manipulations (transpose, submatrix, expansion)
Some methods are readily available to perform common matrix operations such as transposing a matrix, creating a submatrix and expanding a submatrix. 

For full matrices, the <code><b>transposed()</b></code> method is used to transpose a matrix, as shown in the following example:

In [ ]:
mf4_data = emmebank.matrix("mf4").get_data()
transposed_mf4_data = mf4_data.transposed()

To verify the transpose operation, we can query the cell (7,5), which should now contain the same value from cell (5,7) in the original matrix:

In [ ]:
transposed_mf4_data.get(7, 5) == mf4_data.get(5, 7)

... or check that all cells `(p,q)` in the transposed data match `(q,p)` in the original data:

In [ ]:
all_match = True
for p in zones:
    for q in zones:
        if transposed_mf4_data.get(p, q) != mf4_data.get(q, p):
            all_match = False
all_match

Creating submatrices is possible using the <code><b>submatrix()</b></code> method on a <code><b>MatrixData</b></code> object. To create a submatrix, a list of indices (or two lists for full matrices) is required. 

We will create a submatrix containing only zones 1, 5 and 10 using the <code>new_mat</code> full matrix created earlier, which has zones 1-154:

In [ ]:
sub_indices = [[1,5,10],[1,5,10]]
new_submatrix = mf4_data.submatrix(sub_indices)

print "New submatrix indices=%s" % str(new_submatrix.indices)

It is also possible to expand an existing matrix, using the <code><b>expand()</b></code> method. Similarly to <code><b>submatrix()</b></code>, <code><b>expand()</b></code> takes a list of indices, which must be a superset of the matrix data indices. It also takes an optional `default_value` parameter, which sets the initial value for the new O-D pairs added to the matrix. The following example takes the <code>new_mat</code> full matrix and expands it to 200 zones instead of 154, with a value of 5 for every new O-D pair:

In [ ]:
mf4_expanded = mf4_data.expand([(range(1,201)),(range(1,201))], 5)

print "Expanded matrix indices=%s" % str(mf4_expanded.indices)
print "The expanded matrix value at O-D=(200,200) is %s " % str(mf4_expanded.get(200,200))

## 3.4 Working with NumPy arrays
The Matrix API provides direct and fast access to the <a href="http://www.numpy.org/">NumPy</a> Python library which can be used to perform a variety of matrix manipulations. Working with NumPy arrays should be preferred for any operation which can be vectorized as the calculation would be much faster. 

An array based operation can be vectorized if none of the calculation of the array requires to know the result of other calculations from the array.

In [ ]:
matrix_mf5 = emmebank.matrix("mf5")
mf5_data = matrix_mf5.get_numpy_data()

It is also possible to convert a <code><b>MatrixData</b></code> object to a NumPy array using the <code><b>to_numpy()</b></code> method. However, it is much more efficient to create a NumPy array directly from an Emme matrix using the <code><b>get_numpy_data()</b></code> method, as shown in the following example: 

In [ ]:
%%timeit -n 10 
mf5_data = matrix_mf5.get_numpy_data()

In [ ]:
%%timeit -n 10
mf5_data = matrix_mf5.get_data().to_numpy()

Individual cells in NumPy arrays are accessed using dictionary-like indexing based on the element positioning. It is important to note that NumPy arrays do not have a notion of zone IDs. Accessing a specific O-D pair value would require to use the zone index provided with the scenario object:

In [ ]:
zone_index = {}
for index, zone_number in enumerate(scenario.zone_numbers):
    zone_index[zone_number] = index
    
zone_index

In [ ]:
mf5_data[zone_index[77]][zone_index[55]]

It is also possible to modify NumPy array values through assignment. In the following example the matrix data (for every O-D value) is multiplied by 2. Compare to the earlier example with the __`MatrixData`__ <code>mf5</code> done with a double `for` loop.

In [ ]:
mf5_data *= 2
mf5_data[zone_index[77]][zone_index[55]]

Standard Python arithmetic syntax can be used for array-based computations with NumPy.

NumPy provides many tools for array-based computations, and general matrix linear algebra. For example, the following calculates a disutility value from the negative exponential of the travel time in matrix `mf5`.

In [ ]:
import numpy as _numpy
disutil = _numpy.exp(-0.015*mf5_data)

Or can be used for matrix summaries / statistics. For example, the following displays a few statistics about the values in <code>mf5_data</code>.

In [ ]:
mf5_data.size, mf5_data.sum(), mf5_data.min(), mf5_data.max()

NumPy array data can be written to Emmebank using the <code>Matrix.<b>set_numpy_data()</b></code> method. For example, to write the modified NumPy array back to Emme matrix <code>mf5</code>, use the following code:

In [ ]:
matrix_mf5.set_numpy_data(mf5_data)

See the <a href="http://www.numpy.org/">NumPy reference</a> for more. The <a href="https://www.numpy.org/devdocs/user/quickstart.html">tutorial</a> is a good place to start for an introduction.

Note that the modeller tool _Matrix calculator_ can be faster for most matrix expressions as it is multi-threaded. NumPy can, however, provide a low-level interface for a high degree of customization and optimization. 

## 3.5 Importing and exporting to CSV format
To export matrix data to Comma Separated Values (CSV) format, the <code><b>to_csv()</b></code> method is used. In the following example, the matrix data contained in the variable <code>mf4_data</code> is exported to the file <code>mf4_export.csv</code>, in the <i>Database</i> folder of the project.

__Note that there are many Modeller tools which let you export matrices in various format, including OMX, squared CSV etc.__

In [ ]:
import inro.emme.matrix as _matrix
import os

database_path = os.path.dirname(emmebank.path)
export_path = os.path.join(database_path, "mf4_export.csv")
_matrix.to_csv(mf4_data, export_path)

Note that the <code><b>to_csv()</b></code> method requires a <code><b>MatrixData</b></code> object as the first argument. NumPy arrays must be converted to <code><b>MatrixData</b></code> objects prior to export to use this method.

We can open the file and look at its format. The format is a squared table with origins on rows, destination on the columns and the value at the corresponding location.

In [ ]:
with open(export_path) as matrix_file:
    print matrix_file.read()

The <code><b>from_csv()</b></code> method imports matrix data from a CSV format file and converts it to a <code><b>MatrixData</b></code> object, as shown in the following example:

In [ ]:
imported_mat = _matrix.from_csv(export_path)

Note that the first row and column of the file must contain the row and column indices respectively. The matrix data may be full (two-dimensional) or origin or destination (vector). The delimiter is any one-character string; default is a comma. Once this matrix data is loaded, it can be published to a Matrix in the Emmebank using one of the workflow we have seen previously.

In [ ]:
next_available_id = emmebank.available_matrix_identifier('FULL')
new_matrix = emmebank.create_matrix(next_available_id)
new_matrix.set_data(imported_mat)

## 3.6 Additional Example part 1: Modify values of a matrix by district
Consider the example below to multiply the existing matrix
    values by 2 for a subset of zones in ga1, ga2, or ga3:
</p>


<pre>scenario = my_modeller.scenario
emmebank = my_modeller.emmebank
matrix_mf1 = emmebank.matrix("mf1")
matrix_data = matrix_mf1.get_data(scenario.id)

partition_data = emmebank.partition("ga").get_data()

zones = []
for z in scenario.zone_numbers:
    if partition_data.get(z) in (1, 2, 3):
        zones.append(z)

for p in zones:
    for q in zones:
        value = matrix_data.get(p, q)
        matrix_data.set(p, q, value * 2)
</pre>


<p>
   Modify this example to also multiply by 3 the values <i>not</i> in ga1, ga2, or ga3.
   Hint: generate two lists of zone IDs using the partition data.
</p>
<p>See the Matrix API in the Emme API Reference for a full description of methods and properties.</p>

In [ ]:
scenario = modeller.scenario
emmebank = modeller.emmebank
matrix_mf1 = emmebank.matrix("mf1")
matrix_data = matrix_mf1.get_data(scenario)

partition_data = emmebank.partition("ga").get_data()

In [ ]:
zones_in_partition = []
zones_not_in_partition = []

for z in scenario.zone_numbers:
    if partition_data.get(z) in (1, 2, 3):
        zones_in_partition.append(z)
    else:
        zones_not_in_partition.append(z)

for p in zones_in_partition:
    for q in zones_in_partition:
        value = matrix_data.get(p, q)
        matrix_data.set(p, q, value * 2)

for p in zones_not_in_partition:
    for q in zones_not_in_partition:
        value = matrix_data.get(p, q)
        matrix_data.set(p, q, value * 3)      

## 3.7 Additional Example part 2: Compute weighted avg. travel time by OD pair
In our model there is both transit and traffic demand: 

- traffic demand stored in `mf1` and the associated travel times stored in `mf4`
- transit demand stored in `mf2` and the associated travel times stored in `mf5`

Compute the weighted average travel time by OD pair using the matrix Numpy interface. The formula to implement is as follows:

`weighted_average_travel_time = (mf1 * mf4 + mf2 * mf5) / (mf1 + mf2)`

The division by 0 case will raise a warning. We can handle it using the numpy.`nan_to_num()` function to transform NaN values into zeros.

Store the result in a new matrix with ID  _mf13_

In [ ]:
# Access the matrices as numpy arrays
mf1_data = emmebank.matrix('mf1').get_numpy_data()
mf2_data = emmebank.matrix('mf2').get_numpy_data()
mf4_data = emmebank.matrix('mf4').get_numpy_data()
mf5_data = emmebank.matrix('mf5').get_numpy_data()

In [ ]:
# Compute the weighted average travel time
weighted_average_travel_time = (mf1_data * mf4_data + mf2_data * mf5_data) / (mf1_data + mf2_data)

In [ ]:
# Get rid of the NaN values resulting from the division by 0
import numpy as np
weighted_average_travel_time = np.nan_to_num(weighted_average_travel_time)

In [ ]:
# Create the new matrix to store the result of the calculation
mf13 = emmebank.matrix('mf13')
if not mf13:
    mf13 = emmebank.create_matrix('mf13')

mf13.name = 'weighted_avg_tt'
mf13.description = 'Weighted average travel time'

In [ ]:
# Publish the result
mf13.set_numpy_data(weighted_average_travel_time)